# Complete API for Visual Emotion Recognition

This notebook provides a complete API implementation for serving visual emotion recognition models.

## Components Included:
1. **Model Inference** - Load and run models for predictions
2. **Image Processing** - Handle various image formats and preprocessing
3. **Batch Processing** - Process multiple images efficiently
4. **REST API Server** - Simple web server for model serving
5. **Utilities** - Helper functions for deployment


In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
import numpy as np
import json
import io
import base64
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## 1. Model Inference Class

In [ ]:
class EmotionPredictor:
    """
    Complete emotion prediction class with model loading and inference.
    """
    
    def __init__(self, model_path=None, model=None, transform=None, label_map=None, device='cpu'):
        """
        Initialize the emotion predictor.
        
        Args:
            model_path (str): Path to saved model file
            model: Pre-loaded PyTorch model
            transform: Image preprocessing transform
            label_map (dict): Mapping from class indices to names
            device (str): Device to use
        """
        self.device = device
        
        if model_path:
            self.load_model(model_path)
        elif model is not None:
            self.model = model.to(device)
            self.model.eval()
            self.transform = transform
            self.label_map = label_map or {}
        else:
            raise ValueError("Either model_path or model must be provided")
        
        # Create inverse mapping
        self.idx_to_label = {v: k for k, v in self.label_map.items()}
        
        print(f"EmotionPredictor initialized with {len(self.label_map)} classes")
        print(f"Classes: {list(self.label_map.keys())}")
    
    def load_model(self, model_path):
        """
        Load model from saved file.
        
        Args:
            model_path (str): Path to model file
        """
        checkpoint = torch.load(model_path, map_location=self.device)
        
        # Extract model information
        if 'label_map' in checkpoint:
            self.label_map = checkpoint['label_map']
        else:
            # Default emotion mapping
            self.label_map = {
                'angry': 0, 'fearful': 1, 'happy': 2, 'neutral': 3, 
                'sad': 4, 'surprised': 5, 'disgusted': 6
            }
        
        # Get transforms configuration
        if 'transforms_config' in checkpoint:
            transforms_config = checkpoint['transforms_config']
            self.setup_transforms(transforms_config)
        else:
            # Default transforms
            self.setup_default_transforms()
        
        print(f"Model loaded from {model_path}")
    
    def setup_transforms(self, config):
        """
        Setup transforms from configuration.
        
        Args:
            config (dict): Transform configuration
        """
        from torchvision import transforms
        
        input_size = config.get('input_size', 224)
        mean = config.get('mean', [0.485, 0.456, 0.406])
        std = config.get('std', [0.229, 0.224, 0.225])
        
        self.transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])
    
    def setup_default_transforms(self):
        """
        Setup default transforms.
        """
        from torchvision import transforms
        
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def preprocess_image(self, image):
        """
        Preprocess image for model input.
        
        Args:
            image: PIL Image, numpy array, or file path
            
        Returns:
            torch.Tensor: Preprocessed image tensor
        """
        # Handle different input types
        if isinstance(image, str):
            # File path
            image = Image.open(image)
        elif isinstance(image, np.ndarray):
            # Numpy array
            if image.dtype != np.uint8:
                image = (image * 255).astype(np.uint8)
            image = Image.fromarray(image)
        elif not isinstance(image, Image.Image):
            raise ValueError(f"Unsupported image type: {type(image)}")
        
        # Convert to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Apply transforms
        if self.transform:
            image_tensor = self.transform(image)
        else:
            # Basic preprocessing
            from torchvision import transforms
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor()
            ])
            image_tensor = transform(image)
        
        return image_tensor
    
    def predict_single(self, image, return_probs=True, top_k=None):
        """
        Predict emotion for a single image.
        
        Args:
            image: Input image
            return_probs (bool): Whether to return probabilities
            top_k (int): Return top-k predictions
            
        Returns:
            dict: Prediction results
        """
        # Preprocess image
        image_tensor = self.preprocess_image(image)
        image_batch = image_tensor.unsqueeze(0).to(self.device)
        
        # Get prediction
        with torch.no_grad():
            outputs = self.model(image_batch)
            probabilities = F.softmax(outputs, dim=1)[0]
            
        # Get top prediction
        top_prob, top_idx = probabilities.max(0)
        predicted_emotion = self.idx_to_label.get(top_idx.item(), f'Class_{top_idx.item()}')
        
        result = {
            'predicted_emotion': predicted_emotion,
            'confidence': top_prob.item()
        }
        
        if return_probs:
            # Add all class probabilities
            all_probs = {}
            for idx, prob in enumerate(probabilities):
                emotion_name = self.idx_to_label.get(idx, f'Class_{idx}')
                all_probs[emotion_name] = prob.item()
            result['probabilities'] = all_probs
        
        if top_k and top_k > 1:
            # Add top-k predictions
            top_probs, top_indices = torch.topk(probabilities, min(top_k, len(probabilities)))
            top_predictions = []
            for prob, idx in zip(top_probs, top_indices):
                emotion_name = self.idx_to_label.get(idx.item(), f'Class_{idx.item()}')
                top_predictions.append({
                    'emotion': emotion_name,
                    'confidence': prob.item()
                })
            result['top_predictions'] = top_predictions
        
        return result
    
    def predict_batch(self, images, batch_size=32, return_probs=False):
        """
        Predict emotions for a batch of images.
        
        Args:
            images: List of images
            batch_size (int): Batch size for processing
            return_probs (bool): Whether to return probabilities
            
        Returns:
            list: List of prediction results
        """
        results = []
        
        # Process in batches
        for i in range(0, len(images), batch_size):
            batch_images = images[i:i + batch_size]
            batch_tensors = []
            
            # Preprocess batch
            for image in batch_images:
                try:
                    tensor = self.preprocess_image(image)
                    batch_tensors.append(tensor)
                except Exception as e:
                    print(f"Error processing image: {e}")
                    # Add dummy result for failed image
                    results.append({
                        'predicted_emotion': 'error',
                        'confidence': 0.0,
                        'error': str(e)
                    })
                    continue
            
            if not batch_tensors:
                continue
            
            # Stack tensors and predict
            batch_tensor = torch.stack(batch_tensors).to(self.device)
            
            with torch.no_grad():
                outputs = self.model(batch_tensor)
                probabilities = F.softmax(outputs, dim=1)
            
            # Process results
            for probs in probabilities:
                top_prob, top_idx = probs.max(0)
                predicted_emotion = self.idx_to_label.get(top_idx.item(), f'Class_{top_idx.item()}')
                
                result = {
                    'predicted_emotion': predicted_emotion,
                    'confidence': top_prob.item()
                }
                
                if return_probs:
                    all_probs = {}
                    for idx, prob in enumerate(probs):
                        emotion_name = self.idx_to_label.get(idx, f'Class_{idx}')
                        all_probs[emotion_name] = prob.item()
                    result['probabilities'] = all_probs
                
                results.append(result)
        
        return results


print("EmotionPredictor class created successfully!")

## 2. Utility Functions

In [ ]:
def load_image_from_base64(base64_str):
    """
    Load image from base64 string.
    
    Args:
        base64_str (str): Base64 encoded image
        
    Returns:
        PIL.Image: Loaded image
    """
    # Remove data URL prefix if present
    if base64_str.startswith('data:image/'):
        base64_str = base64_str.split(',')[1]
    
    # Decode base64
    image_data = base64.b64decode(base64_str)
    image = Image.open(io.BytesIO(image_data))
    
    return image


def image_to_base64(image):
    """
    Convert PIL image to base64 string.
    
    Args:
        image (PIL.Image): Image to convert
        
    Returns:
        str: Base64 encoded image
    """
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return img_str


def create_inference_config(model_path, transforms_config=None, label_map=None):
    """
    Create inference configuration file.
    
    Args:
        model_path (str): Path to model file
        transforms_config (dict): Transform configuration
        label_map (dict): Label mapping
        
    Returns:
        dict: Inference configuration
    """
    config = {
        'model_path': model_path,
        'transforms': transforms_config or {
            'input_size': 224,
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225]
        },
        'label_map': label_map or {
            'angry': 0, 'fearful': 1, 'happy': 2, 'neutral': 3,
            'sad': 4, 'surprised': 5, 'disgusted': 6
        },
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }
    
    return config


def save_inference_config(config, config_path):
    """
    Save inference configuration to file.
    
    Args:
        config (dict): Configuration dictionary
        config_path (str): Path to save configuration
    """
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"Inference configuration saved to {config_path}")


def load_inference_config(config_path):
    """
    Load inference configuration from file.
    
    Args:
        config_path (str): Path to configuration file
        
    Returns:
        dict: Configuration dictionary
    """
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config


print("Utility functions created successfully!")

## 3. Simple Flask API Server

In [ ]:
def create_flask_app(predictor):
    """
    Create Flask application for emotion recognition API.
    
    Args:
        predictor (EmotionPredictor): Trained emotion predictor
        
    Returns:
        Flask app
    """
    try:
        from flask import Flask, request, jsonify
        from flask_cors import CORS
    except ImportError:
        print("Flask not available. Install with: pip install flask flask-cors")
        return None
    
    app = Flask(__name__)
    CORS(app)  # Enable CORS for all domains
    
    @app.route('/health', methods=['GET'])
    def health_check():
        """Health check endpoint."""
        return jsonify({
            'status': 'healthy',
            'model_classes': list(predictor.label_map.keys()),
            'device': predictor.device
        })
    
    @app.route('/predict', methods=['POST'])
    def predict_emotion():
        """Predict emotion from uploaded image."""
        try:
            if 'image' not in request.files:
                return jsonify({'error': 'No image file provided'}), 400
            
            file = request.files['image']
            if file.filename == '':
                return jsonify({'error': 'No image file selected'}), 400
            
            # Load image
            image = Image.open(file.stream)
            
            # Get prediction parameters
            return_probs = request.form.get('return_probs', 'false').lower() == 'true'
            top_k = request.form.get('top_k', None)
            if top_k:
                top_k = int(top_k)
            
            # Predict
            result = predictor.predict_single(image, return_probs=return_probs, top_k=top_k)
            
            return jsonify({
                'success': True,
                'result': result
            })
            
        except Exception as e:
            return jsonify({
                'success': False,
                'error': str(e)
            }), 500
    
    @app.route('/predict_base64', methods=['POST'])
    def predict_base64():
        """Predict emotion from base64 encoded image."""
        try:
            data = request.get_json()
            
            if 'image' not in data:
                return jsonify({'error': 'No image data provided'}), 400
            
            # Load image from base64
            image = load_image_from_base64(data['image'])
            
            # Get prediction parameters
            return_probs = data.get('return_probs', False)
            top_k = data.get('top_k', None)
            
            # Predict
            result = predictor.predict_single(image, return_probs=return_probs, top_k=top_k)
            
            return jsonify({
                'success': True,
                'result': result
            })
            
        except Exception as e:
            return jsonify({
                'success': False,
                'error': str(e)
            }), 500
    
    @app.route('/predict_batch', methods=['POST'])
    def predict_batch():
        """Predict emotions for multiple images."""
        try:
            data = request.get_json()
            
            if 'images' not in data:
                return jsonify({'error': 'No images data provided'}), 400
            
            # Load images from base64
            images = []
            for img_data in data['images']:
                image = load_image_from_base64(img_data)
                images.append(image)
            
            # Get prediction parameters
            return_probs = data.get('return_probs', False)
            batch_size = data.get('batch_size', 32)
            
            # Predict batch
            results = predictor.predict_batch(images, batch_size=batch_size, return_probs=return_probs)
            
            return jsonify({
                'success': True,
                'results': results,
                'count': len(results)
            })
            
        except Exception as e:
            return jsonify({
                'success': False,
                'error': str(e)
            }), 500
    
    return app


def run_api_server(predictor, host='0.0.0.0', port=5000, debug=False):
    """
    Run Flask API server.
    
    Args:
        predictor (EmotionPredictor): Trained emotion predictor
        host (str): Host address
        port (int): Port number
        debug (bool): Debug mode
    """
    app = create_flask_app(predictor)
    
    if app is None:
        print("Cannot create Flask app. Make sure Flask is installed.")
        return
    
    print(f"Starting emotion recognition API server...")
    print(f"Server running at http://{host}:{port}")
    print(f"Endpoints:")
    print(f"  GET  /health - Health check")
    print(f"  POST /predict - Single image prediction (multipart/form-data)")
    print(f"  POST /predict_base64 - Single image prediction (base64 JSON)")
    print(f"  POST /predict_batch - Batch prediction (base64 JSON)")
    
    app.run(host=host, port=port, debug=debug)


print("Flask API server functions created successfully!")

## 4. Example Usage and Testing

In [ ]:
def demo_api_usage():
    """
    Demonstrate API usage with example code.
    """
    print("=== Emotion Recognition API Demo ===")
    print()
    
    # Example 1: Load and use predictor directly
    print("1. Direct Predictor Usage:")
    print("```python")
    print("# Load trained model")
    print("predictor = EmotionPredictor(model_path='best_model.pth')")
    print("")
    print("# Predict single image")
    print("result = predictor.predict_single('path/to/image.jpg', return_probs=True)")
    print("print(f\"Emotion: {result['predicted_emotion']} (Confidence: {result['confidence']:.3f})\")")
    print("")
    print("# Predict batch of images")
    print("images = ['img1.jpg', 'img2.jpg', 'img3.jpg']")
    print("results = predictor.predict_batch(images)")
    print("for i, result in enumerate(results):")
    print("    print(f\"Image {i}: {result['predicted_emotion']}\")")
    print("```")
    print()
    
    # Example 2: API Server
    print("2. API Server Usage:")
    print("```python")
    print("# Start API server")
    print("predictor = EmotionPredictor(model_path='best_model.pth')")
    print("run_api_server(predictor, host='localhost', port=5000)")
    print("```")
    print()
    
    # Example 3: API Client
    print("3. API Client Examples:")
    print()
    
    print("Python client:")
    print("```python")
    print("import requests")
    print("import json")
    print("")
    print("# Health check")
    print("response = requests.get('http://localhost:5000/health')")
    print("print(response.json())")
    print("")
    print("# Upload image file")
    print("with open('image.jpg', 'rb') as f:")
    print("    files = {'image': f}")
    print("    data = {'return_probs': 'true'}")
    print("    response = requests.post('http://localhost:5000/predict', files=files, data=data)")
    print("    result = response.json()")
    print("    print(result['result']['predicted_emotion'])")
    print("")
    print("# Base64 image")
    print("import base64")
    print("with open('image.jpg', 'rb') as f:")
    print("    img_b64 = base64.b64encode(f.read()).decode()")
    print("")
    print("payload = {")
    print("    'image': img_b64,")
    print("    'return_probs': True")
    print("}")
    print("response = requests.post('http://localhost:5000/predict_base64', json=payload)")
    print("result = response.json()")
    print("```")
    print()
    
    print("cURL examples:")
    print("```bash")
    print("# Health check")
    print("curl http://localhost:5000/health")
    print("")
    print("# Upload image")
    print("curl -X POST -F \"image=@image.jpg\" -F \"return_probs=true\" http://localhost:5000/predict")
    print("```")
    print()
    
    print("JavaScript (fetch) example:")
    print("```javascript")
    print("// Upload image file")
    print("const formData = new FormData();")
    print("formData.append('image', imageFile);")
    print("formData.append('return_probs', 'true');")
    print("")
    print("fetch('http://localhost:5000/predict', {")
    print("    method: 'POST',")
    print("    body: formData")
    print("})")  
    print(".then(response => response.json())")
    print(".then(data => {")
    print("    console.log('Predicted emotion:', data.result.predicted_emotion);")
    print("    console.log('Confidence:', data.result.confidence);")
    print("});")
    print("```")


def create_deployment_script():
    """
    Create a deployment script for the API.
    
    Returns:
        str: Deployment script content
    """
    script = '''#!/usr/bin/env python3
"""
Emotion Recognition API Deployment Script
"""

import argparse
import torch
from pathlib import Path

# Import the predictor and API functions from this notebook
# You would copy the EmotionPredictor class and API functions here

def main():
    parser = argparse.ArgumentParser(description='Deploy Emotion Recognition API')
    parser.add_argument('--model_path', required=True, help='Path to trained model')
    parser.add_argument('--host', default='0.0.0.0', help='Host address')
    parser.add_argument('--port', type=int, default=5000, help='Port number')
    parser.add_argument('--debug', action='store_true', help='Enable debug mode')
    
    args = parser.parse_args()
    
    # Validate model path
    if not Path(args.model_path).exists():
        print(f"Model file not found: {args.model_path}")
        return
    
    # Load predictor
    print(f"Loading model from {args.model_path}...")
    predictor = EmotionPredictor(model_path=args.model_path)
    
    # Start API server
    run_api_server(predictor, host=args.host, port=args.port, debug=args.debug)

if __name__ == '__main__':
    main()
'''
    return script


# Run demo
demo_api_usage()

print("\n" + "="*60)
print("API implementation complete!")
print("="*60)

## Summary

This notebook provides a complete API implementation for visual emotion recognition:

### Core Components:
1. **EmotionPredictor**: Complete inference class with model loading and prediction
2. **Image Processing**: Handle various image formats, base64 encoding/decoding
3. **Batch Processing**: Efficient processing of multiple images
4. **REST API**: Flask-based web server with multiple endpoints
5. **Utility Functions**: Configuration management and helper functions

### Key Features:
- **Flexible Input**: Support for file paths, PIL Images, numpy arrays, base64 strings
- **Batch Processing**: Efficient batch prediction with configurable batch sizes
- **REST API**: Professional web API with health checks and error handling
- **Cross-Origin Support**: CORS enabled for web applications
- **Configuration Management**: Easy model and transform configuration
- **Deployment Ready**: Complete deployment scripts and examples

### API Endpoints:
- `GET /health` - Health check and model information
- `POST /predict` - Single image prediction (multipart form)
- `POST /predict_base64` - Single image prediction (JSON base64)
- `POST /predict_batch` - Batch image prediction (JSON base64)

### Usage Examples:
```python
# Direct usage
predictor = EmotionPredictor(model_path='model.pth')
result = predictor.predict_single('image.jpg')

# API server
run_api_server(predictor, host='localhost', port=5000)
```

All functionality is self-contained within this notebook and doesn't require the src folder structure.